추가/수정 사항
1. 0:ws 로 기사의 앞부분만 뽑아내고 있는데, 좀더 유튜브 스크립트처럼 하려면 여러 위치에서 뽑아야 할 듯
2. 

In [1]:
import os
import sys
import torch
from tqdm import tqdm
import random
import pickle
print(torch.__version__)

1.1.0


# DATASET

In [2]:
import json
def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

In [3]:
os.getcwd()

'/repo/course/sem21_01/youtube_summarizer/jupyter'

In [4]:
data_path = '../dataset/article_dataset/'
news_df = load_jsonl(data_path+'train.jsonl')

Loaded 260697 records from ../dataset/article_dataset/train.jsonl


In [5]:
# 전처리
# (1) 글자 개수가 너무 작은 경우 없애기 (30글자 이상)
# (2) 문장이 적은 경우 해당 기사 없애기 (10문장 이상)
news_clean = []
for news in news_df:
    news_article = news['article_original']
    if len(news_article) >= 10:
        article_clean = [sent for sent in news_article]
        news_clean.append(article_clean)

# BertSum

In [6]:
import os
os.getcwd()

'/repo/course/sem21_01/youtube_summarizer/jupyter'

In [7]:
# path 추가
sys.path.append('/home/sks/korea_univ/21_1/TA/team_project/youtube_summarizer/src/bertsum')
sys.path.append('/repo/course/sem21_01/youtube_summarizer/src/bertsum')
sys.path.append('/repo/course/sem21_01/youtube_summarizer/src/subtext/src')

In [10]:
from models.data_loader import TextLoader, load_dataset
from backbone import args, ExtTransformerEncoder, ExtSummarizer, WindowEmbedder

In [12]:
# Settings
device = "cpu" if args.visible_gpus == -1 else "cuda"
loader = TextLoader(args, device)

# model setting
# ckpt_path = '/home/sks/korea_univ/21_1/TA/project/video_summarizer/Youtube-Summarizer/src/bertsum/checkpoint/model_step_24000.pt'

checkpoint = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model = ExtSummarizer(args, device, checkpoint)
model.eval()

using cached model
using cached model


FileNotFoundError: [Errno 2] No such file or directory: '/home/sks/korea_univ/21_1/TA/project/video_summarizer/Youtube-Summarizer/src/bertsum/checkpoint/model_step_24000.pt'

# Make Trainingset (dev/test)

In [20]:
max_dt = 50000
df_base = news_clean[:max_dt]

In [21]:
# 셔플링
random.seed(1234)
random.shuffle(df_base)

In [22]:
# hyperparameter
window_size = 3

train_ratio = 0.7
val_ratio = 0.15
test_raio = 0.15

In [23]:
train_len, val_len = int(len(df_base)*train_ratio), int(len(df_base)*val_ratio)
test_len = len(df_base) - (train_len + val_len)
print(f"Train_base: {train_len}, Val_base: {val_len}, Test_base: {test_len}")

Train_base: 35000, Val_base: 7500, Test_base: 7500


In [24]:
train_base = df_base[:train_len]
val_base = df_base[train_len:train_len+val_len]
test_base = df_base[train_len+val_len:]

In [28]:
def make_dataset(base_dataset=None, y_ratio=0.5, window_size=4):
    '''
    y_data: Mixed article
    n_data: Normal article
    '''
    tot_len = base_dataset.__len__()
    y_len = int(tot_len * y_ratio)
    
    y_cands = base_dataset[:y_len]
    n_cands = base_dataset[y_len:]
    
    y_dataset, n_dataset = [], []
    for i in tqdm(range(len(y_cands) - 1), desc='Sampling Y dataset'):
        tmp_article_y = y_cands[i][:window_size] + y_cands[i+1][:window_size]
        y_dataset.append(tmp_article_y)
        
    for j in tqdm(range(len(n_cands)), desc='Sampling N dataset'):
        tmp_article_n = n_cands[j][:window_size*2]
        n_dataset.append(tmp_article_n)
        
    return y_dataset, n_dataset

In [29]:
train_div, train_org = make_dataset(base_dataset=train_base, y_ratio=0.5, window_size=window_size)
val_div, val_org = make_dataset(base_dataset=val_base, y_ratio=0.5, window_size=window_size)
test_div, test_org = make_dataset(base_dataset=test_base, y_ratio=0.5, window_size=window_size)

Sampling N dataset: 100%|██████████| 3750/3750 [00:00<00:00, 681129.40it/s]


# Get Embeddings

In [31]:
embedder = WindowEmbedder(model=model, window_size=window_size, text_loader=loader)

In [32]:
tot_datasets = [train_div, train_org, val_div, val_org, test_div, test_org]
#tot_datasets = [test_org]

tot_embs = []
for dt in tot_datasets:
    tmp_emb = []
    for article in tqdm(dt, desc="working on articles"):
        embedding = embedder.get_embeddings(article)
        if embedding.size()[0] == window_size*2:
            tmp_emb.append(embedding.unsqueeze(0))
    tmp_dt_emb = torch.cat(tmp_emb, dim=0)
    tot_embs.append(tmp_dt_emb)

working on articles: 100%|██████████| 3750/3750 [12:33<00:00,  4.98it/s]


In [37]:
data_save_path = f"/home/sks/korea_univ/21_1/TA/project/video_summarizer/Youtube-Summarizer/src/subtext/dataset/nn_dataset_w{window_size}.pkl"

with open(data_save_path, 'wb') as ww:
    pickle.dump(tot_embs, ww)

In [ ]:
# Load
with open(data_save_path, 'rb') as rr:
    tot_embs = pickle.load(rr)

In [25]:
tot_embs[0].size()

torch.Size([17350, 8, 768])

# Make Dataset (window=3)